# Exploratory Data Analysis of the Kingston Housing Data - Regression Model

## Imports

In [1]:
from sklearn.utils import check_array
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import operator
import statsmodels.formula.api as smf
pd.options.mode.chained_assignment = None 

## Modeling

In [2]:
# It's time to fit our initial model. 
# First prepare the data
data = pd.read_pickle('King_County_House_prices_dataset_EXPLORED.pkl')
train, test = train_test_split(data, test_size = 0.1, random_state = 500)

# We'll do a regression with all factors (minus the 'nonsensical' ones we identified during cleaning/exploration)
# Note which columns we do NOT want to include as an explanatory inside a list ('price' is target, so not expl.)

droplist = ['price', 'id', 'zipcode', 'date']

# Since we will be fitting multiple models we write a function that does this for us and calculates the MAPE
# We provide this function it with the target (as a string), our droplist and the test set
def create_model(target, droplist, test): 
    
    # Create model
    model = f'{target} ~ '+' + '.join(train.columns)+ ' - ' + ' - '.join(droplist)
    results = smf.ols(model, data = train).fit()
    
    # Calculate MAPE
    test['pred'] = results.predict(test)
    test.dropna(inplace = True)
    
    actual = np.array(test[target])
    pred = np.array(test['pred'])
    mape =  (np.mean(np.abs((actual - pred) / actual)) * 100).round(2)
    
    # Return statement
    print(f'''I fitted a model with {len(set([x.split('[')[0] for x in results.params.index]))-1} explanatory variables.\n
The MAPE of the model is {mape}%.\n''')
    
    return results



# Our initial model is ready to go    
results = create_model('price', droplist, test)
print(results.summary())

I fitted a model with 17 explanatory variables.

The MAPE of the model is 22.18%.

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                     1841.
Date:                Fri, 18 Sep 2020   Prob (F-statistic):               0.00
Time:                        09:31:27   Log-Likelihood:            -2.2901e+05
No. Observations:               16935   AIC:                         4.581e+05
Df Residuals:                   16904   BIC:                         4.583e+05
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------

In [3]:
# A MAPE of 22.18 does not seem that bad for a first try, but we can probably do better
# Let's compare the MAPE of that model to the same one, just wothout our selfmade 'topzip' variable
data = pd.read_pickle('King_County_House_prices_dataset_EXPLORED.pkl')
train, test = train_test_split(data, test_size = 0.1, random_state = 500)

droplist = ['price', 'id', 'zipcode', 'date', 'topzip']

results = create_model('price', droplist, test)

I fitted a model with 16 explanatory variables.

The MAPE of the model is 23.86%.



In [4]:
# Even though we weren't sure about the approach we took with the 'topzip' column, 
# the MAPE does seem to react positively to it.
# Maybe if we put every zipcodes in its own category, we can get even better. 

# Let's make some changes to the model, we start with the same base:
data = pd.read_pickle('King_County_House_prices_dataset_EXPLORED.pkl')
droplist = ['price', 'id', 'zipcode', 'date']

# We include zipcode as a categorial factor (and remove topzip, since it is now unneccesary)
data['zipcode'] = data.zipcode.astype('category')
droplist.remove('zipcode')
droplist.append('topzip')

# MAPE seemed to respond positively to categorising certain variables, since we can somewhat get around our 
# restriction in choosing a linear regression model. How about we also do this to 'bathrooms' and 'yr_built':
data['bathrooms'] = data.bathrooms.astype('category')
data['yr_built'] = data.yr_built.astype('category')

# In real estate, the price per square feet is often the preferred method in evaluating if a house is expensive
# Maybe we should change the target variable to price/sqft then and remove sqft_living from the model
data['price_sqft'] = data['price']/data['sqft_living']
droplist.append('price_sqft') # not explanatory (target), thus dropped
droplist.append('sqft_living')

# While we're on it, why only check the absoute values of sqft_living15 and sqft_lot15?
# We may also want to know how our house compares with our neighbours:
data['ratio_sqft_living15'] = data['sqft_living'] / data['sqft_living15']
data['ratio_sqft_lot15'] = data['sqft_lot'] / data['sqft_lot15']


# We are kind of abusing the shortcomings of MAPE here, but since it is the only figure we can use to 
# evaluate the model (according to our assignment), we're going all out with it for now
# Normally we would now take out the original 'sqft_living', 'sqft_living15' and 'sqft_lot15' (and we will in a moment)
# But let's check this model first since the MAPE is probably not going to get any better

train, test = train_test_split(data, test_size = 0.1, random_state = 500)
results = create_model('price_sqft', droplist, test)
print(results.summary())

I fitted a model with 18 explanatory variables.

The MAPE of the model is 15.23%.

                            OLS Regression Results                            
Dep. Variable:             price_sqft   R-squared:                       0.729
Model:                            OLS   Adj. R-squared:                  0.726
Method:                 Least Squares   F-statistic:                     188.3
Date:                Fri, 18 Sep 2020   Prob (F-statistic):               0.00
Time:                        09:31:27   Log-Likelihood:                -92586.
No. Observations:               16935   AIC:                         1.857e+05
Df Residuals:                   16695   BIC:                         1.875e+05
Df Model:                         239                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

In [5]:
# A MAPE of 16.04% is not that bad, considering we did not remove any outliers in our cleaning process
# With the creation of this model we are done with the assignment. 

# Hower we probably have some explanatory variables that do only contribute very litte to a smaller MAPE
# We should remove some of the clutter we introduced in that last model

In [6]:
# Eliminate those factors we did not really see have an impact on price during exploration
droplist.extend(['floors', 'bedrooms', 'condition', 'sqft_basement', 'long', 'lat', 'yr_built'])

# Eliminate all remaining 'square feet' variables because they are already represented in
# those new factors we created in the last model
droplist.extend(['sqft_living15', 'sqft_lot15', 'sqft_lot'])
                 
# Last but not least: These factors do also not seem to influence MAPE that much.
droplist.extend(['lowpricemonth', 'highpricemonth', 'ratio_sqft_lot15'])


train, test = train_test_split(data, test_size = 0.1, random_state = 500)
results = create_model('price_sqft', droplist, test)
print(results.summary())

I fitted a model with 5 explanatory variables.

The MAPE of the model is 16.41%.

                            OLS Regression Results                            
Dep. Variable:             price_sqft   R-squared:                       0.690
Model:                            OLS   Adj. R-squared:                  0.688
Method:                 Least Squares   F-statistic:                     353.9
Date:                Fri, 18 Sep 2020   Prob (F-statistic):               0.00
Time:                        09:31:28   Log-Likelihood:                -95810.
No. Observations:               17308   AIC:                         1.918e+05
Df Residuals:                   17199   BIC:                         1.927e+05
Df Model:                         108                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

In [7]:
# To get that good of a MAPE with only 5 explanatory variables seems pretty impressive.
# R-squared and adjusted R-squared took a hit though, but as far as optimizing the model
# while only looking at MAPE, we accomplished our goal
# With this we can consider the assignment completed